In [ ]:
import numpy as np
import xarray as xr
import xgcm
import seawater as sw

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

### Compute the ocean-ice-heat flux
The vertical heat flux from the ocean to the sea ice is estimated at the ocean surface as is done in the model by  

$OIQ(y) = C_{p} \rho \gamma [\overline{u^{*} (T - T_{f})}]$  

where $\gamma=0.006$ is a heat transfer coefficient and $u^{*}$ a friction velocity between ice and ocean, estimated as $u^{*} = \sqrt{0.0054 (u^{2} + v^{2})}$. The mean and transient components are calculated as $\langle OIQ(y) \rangle = c_{p} \rho \gamma [\overline{\langle u^{*} \rangle (T - T_{f})}]$, where $\langle u^{*} \rangle = \sqrt{0.0054 \langle u \rangle^{2} + \langle v \rangle^{2}}$ and $OIQ' = C_{p} \rho \gamma [\overline{(u^{*})' (T - T_{f})}]$, with $(u^{*})' = \sqrt{0.0054 (u^{'2} + v^{'2})}$.

We compute these term for the different seasons.  
First, we define a function to compute the mean and anomalies.

In [ ]:
def bar_prime_season(var, season):
    varbar = var.groupby('time.month').mean("time")
    varbar_season = var.groupby('time.season').mean("time").sel(season=season)
    varprime = (var.groupby('time.month') - varbar)
    return varbar_season, varprime

Defining some constants

In [ ]:
rho = 1035.
Cp = 3994.
gamma = 0.006

and loading the data, as well as defining the xgxm-grid for interpolation.

In [ ]:
em = xr.open_mfdataset(eddypath + 'eddymask_0201-0300.nc', concat_dim="time", combine="nested",
                                 data_vars='minimal', coords='minimal', 
                                 compat='override').rename({"lon": "XG", "lat": "YG"})
em_cyclones = xr.open_mfdataset(eddypath + 'eddymask_cyclones_0201-0300.nc', concat_dim="time", combine="nested",
                                          data_vars='minimal', coords='minimal', 
                                          compat='override').rename({"lon": "XG", "lat": "YG"})
em_anticyclones = xr.open_mfdataset(eddypath + 'eddymask_anticyclones_0201-0300.nc', concat_dim="time", combine="nested",
                                              data_vars='minimal', coords='minimal', 
                                              compat='override').rename({"lon": "XG", "lat": "YG"})
ds = xr.open_zarr(path + "zarr_Diags/output.5d.zarr")
metrics = {
    ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
    ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
    ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
    ('X', 'Y'): ['rAw', 'rAs', 'rA', 'rAz'] # Areas in x-y plane
    }
grid = xgcm.Grid(ds, periodic=["X"], metrics=metrics)

For each decade, we loop over the four seasons.

In [ ]:
a = np.arange(201, 300, 10)
b = np.arange(210, 301, 10)
# loop over the decades
for yy1, yy2 in zip(a, b):
    y1 = f"{yy1:04}"
    y2 = f"{yy2:04}"
    print(y1,"to",y2)
    # extract the eddymasks and data for the respective decade
    eddymask = em.sel(time=slice(y1 + '-01-01', y2 + '-12-30')).eddymask_binary.squeeze()
    eddymask_cyclones = em_cyclones.sel(time=slice(y1 + '-01-01', y2 + '-12-30')).eddymask_cyclones_binary.squeeze()
    eddymask_anticyclones = em_anticyclones.sel(time=slice(y1 + '-01-01', y2 + '-12-30')).eddymask_anticyclones_binary.squeeze()
    data = ds.sel(time=slice(y1 + "-01-01", y2 + "-12-30"))
    # create timestamp for dataset to be saved to disk
    savetime = data.time.isel(time=slice(int(len(data.time) / 2), int(len(data.time) / 2) + 1))
    # extract the surace ocean level
    data_surf = data.isel(Z=0)
    # loop over the four seasons
    for season in ["DJF", "MAM", "JJA", "SON"]:
        # compute mean and anomalies
        u_bar, u_prime = bar_prime_season(data_surf.UVEL, season)
        v_bar, v_prime = bar_prime_season(data_surf.VVEL, season)
        # calculate the surface speed to compute u^star
        speed = (grid.interp(data_surf.UVEL ** 2., "Y", boundary="extend") + grid.interp(data_surf.VVEL ** 2., "X", boundary="extend")) ** 0.5
        speed_bar = (grid.interp(u_bar ** 2., "Y", boundary="extend") + grid.interp(v_bar ** 2., "X", boundary="extend")) ** 0.5
        speed_prime = (grid.interp(u_prime ** 2., "Y", boundary="extend") + grid.interp(v_prime ** 2., "X", boundary="extend")) ** 0.5
        ustar = (0.0054 * (speed ** 2.)) ** 0.5
        ustar_bar = (0.0054 * (speed_bar ** 2.)) ** 0.5
        ustar_prime = (0.0054 * (speed_prime ** 2.)) ** 0.5
        # substract reference temperature from model temperature (heat flux to the ice is defined based on local freezing point temp.)
        tref = sw.eos80.fp(data_surf.SALT, 1.065)
        T = data_surf.THETA - tref
        # compute the different components of the heat flux
        oiq_total = rho * Cp * gamma * ustar * grid.interp(T * data_surf.SIarea, 
                                                           ("X", "Y"), boundary="extend")
        oiq_bar = rho * Cp * gamma * (ustar_bar * grid.interp(T * data_surf.SIarea, 
                                                              ("X", "Y"), boundary="extend")).groupby('time.season').mean("time").sel(season=season)
        oiq_prime = rho * Cp * gamma * ustar_prime * grid.interp(T * data_surf.SIarea, 
                                                                 ("X", "Y"), boundary="extend")
        oiq_prime_eddy = rho * Cp * gamma * ustar_prime * grid.interp(T * data_surf.SIarea, 
                                                                      ("X", "Y"), boundary="extend") * eddymask
        oiq_prime_eddy_cyclones = rho * Cp * gamma * ustar_prime * grid.interp(T * data_surf.SIarea, 
                                                                               ("X", "Y"), boundary="extend") * eddymask_cyclones
        oiq_prime_eddy_anticyclones = rho * Cp * gamma * ustar_prime * grid.interp(T * data_surf.SIarea, 
                                                                                   ("X", "Y"), boundary="extend") * eddymask_anticyclones
        #
        OIQtotal = (oiq_total 
                    * data_surf.dxV * data_surf.dyU).groupby("time.season").mean("time").sel(season=season).sum("XG")
        OIQbar = (oiq_bar 
                  * data_surf.dxV * data_surf.dyU).sum("XG")
        OIQprime = (oiq_prime 
                    * data_surf.dxV * data_surf.dyU).groupby("time.season").mean("time").sel(season=season).sum("XG")
        OIQ_prime_eddy = (oiq_prime_eddy 
                          * data_surf.dxV * data_surf.dyU).groupby("time.season").mean("time").sel(season=season).sum("XG")
        OIQ_prime_eddy_cyclones = (oiq_prime_eddy_cyclones 
                                   * data_surf.dxV * data_surf.dyU).groupby("time.season").mean("time").sel(season=season).sum("XG")
        OIQ_prime_eddy_anticyclones = (oiq_prime_eddy_anticyclones 
                                       * data_surf.dxV * data_surf.dyU).groupby("time.season").mean("time").sel(season=season).sum("XG")
        #
        OIQ = xr.Dataset(coords={"time": savetime, "YG": data.YG})
        OIQ["OIQtotal"] = xr.DataArray(OIQtotal.values[None, :], dims=("time", "YG"))
        OIQ["OIQbar"] = xr.DataArray(OIQbar.values[None, :], dims=("time", "YG"))
        OIQ["OIQprime"] = xr.DataArray(OIQprime.values[None, :], dims=("time", "YG"))
        OIQ["OIQprimeeddy"] = xr.DataArray(OIQ_prime_eddy.values[None, :], dims=("time", "YG"))
        OIQ["OIQprimeeddy_cyclones"] = xr.DataArray(OIQ_prime_eddy_cyclones.values[None, :], dims=("time", "YG"))
        OIQ["OIQprimeeddy_anticyclones"] = xr.DataArray(OIQ_prime_eddy_anticyclones.values[None, :], dims=("time", "YG"))
        #
        OIQ.to_netcdf(path + "post/OIQ." + season + "." + y1 + "_" + y2 + ".nc")